### Setup

In [1]:
# import pandas
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf

# exchange codes http://finabase.blogspot.com/2014/09/interantional-stock-exchange-codes-for.html

path = './Compustat data.csv.nosync.csv'
df = pd.read_csv(path, sep=',')

/Users/vini2001/miniforge3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8,10,12,16,26,30,945,946,948,949,950,955,965,969,970,975) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Configurations

In [2]:
year_start = 2016
year_end = 2017
min_tickers_to_select_row = 4000
only_tickers_with_fiscal_year_ending_at = 12

### Code


#### Remove some exchanges and drop rows without market value

In [13]:
print(len(df))
# drop all df rows where exchg is [0, 1, 2, 19]
df = df[df['exchg'] != 0]
df = df[df['exchg'] != 1]
df = df[df['exchg'] != 2]
df = df[df['exchg'] != 13]
df = df[df['exchg'] != 19]

# canada
df = df[df['exchg'] != 7]
df = df[df['exchg'] != 8]
df = df[df['exchg'] != 9]

# drop rows without market value
df = df[df['mkvalt'].notna()]
print(len(df))

tickers_to_exclude = ['NE', 'EVTL', 'VAL', 'CHK', 'GPOR', 'FYBR', 'CBL', 'TUEM', 'DTLA.P']
df = df[~df['tic'].isin(tickers_to_exclude)]
print(len(df))

127868
123621
123621


#### Aux functions

In [14]:

path_items_descriptions = './items_descriptions.csv'
df_items_descriptions = pd.read_csv(path_items_descriptions, sep=',')
def get_description(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['ItemName'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['ItemDesc']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def get_file_tag(item):
    item = item.upper()
    
    vals = df_items_descriptions[
        (df_items_descriptions['FileTag'] == item ) | 
        (df_items_descriptions['ItemHdr'] == item) |
        (df_items_descriptions['XpressfeedMnemonic'] == item) 
    ]['FileTag']
    return vals.values[0] if len(vals) > 0 else 'Not found'

def val_for_tic_at_year(tic, year, month=None):
    msft = yf.Ticker(tic)
    hist = msft.history(period="max")
    hist.to_csv('hist.csv')
    hist = pd.read_csv('hist.csv')
    dft = df[(df['tic'] == tic) & (df['fyear'] == year)]
    date = dft['datadate'].values[0]
    if month != None : 
        monthN = month if month <= 12 else month - 12
        date = f'{year if month <= 12 else year+1}{monthN if monthN > 9 else f"0{monthN}" }30'

    date = pd.to_datetime(date, format='%Y%m%d')
    close = None
    for i in range(0, 5):
        dateStr = date.strftime('%Y-%m-%d')
        valsDay = hist[hist['Date'] == dateStr]
        if len(valsDay) > 0:
            close = valsDay['Close'].values[0]
            break
        else:
            date = date + pd.DateOffset(days=1)
    return close

def market_caps_for_year(year):
    dfyear = df[df['fyear'] == year]
    # return df with mkvalt and tic
    dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
    dfyear = dfyear[['tic', 'mkvalt', 'price']]
    #drop nans
    dfyear = dfyear.dropna()
    return dfyear



#### Most frequent columns

In [15]:
columns = df.columns.tolist()
print(columns)

df.head()


tickers = df['tic']
# get set of unique tickers
tickers = set(tickers)
print(f'{len(tickers)} tickers')

path = 'tickers.csv'
df_tickers = pd.DataFrame(tickers)
df_tickers.to_csv(path, index=False)

df['fyear']

['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'act', 'adpac', 'aedi', 'afudcc', 'afudci', 'aldo', 'am', 'amc', 'amdc', 'amgw', 'ano', 'ao', 'aocidergl', 'aociother', 'aocipen', 'aocisecgl', 'aodo', 'aol2', 'aoloch', 'aox', 'ap', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqc', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'at', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'bcltbl', 'bcnlr', 'bcrbl', 'bct', 'bctbl', 'bctr', 'billexce', 'bkvlps', 'bltb

81        1998.0
82        1999.0
83        2000.0
84        2001.0
85        2002.0
           ...  
593049    2021.0
593052    2021.0
593056    2021.0
593060    2021.0
593062    2021.0
Name: fyear, Length: 123621, dtype: float64

In [16]:

def most_frequent_columns(df):
    map_count = {}
    for c in columns:
        map_count[c] = df[c].describe()['count']

    # sort by count
    map_count = {k: v for k, v in sorted(map_count.items(), key=lambda item: item[1], reverse=True)}

    columns_data = []
    for k, v in map_count.items():
        columns_data.append([k, v, get_description(k), get_file_tag(k)])

    df_columns = pd.DataFrame(columns_data, columns=['Column', 'Count', 'Description', 'FileTag'])
    return df_columns

df_columns = most_frequent_columns(df)

path = './columns.csv'
df_columns.to_csv(path, index=False)
df_columns.head()

,Column,Count,Description,FileTag
0,gvkey,123621.0,Global Company Key - Key Selection,Multiple Files
1,datadate,123621.0,Data Date,Multiple Files
2,fyear,123621.0,Data Year - Fiscal,DES_ANN
3,indfmt,123621.0,Not found,Not found
4,consol,123621.0,Level of Consolidation,CCM_FILEDATE


#### Select rows for year_start

In [17]:
rows_start = df[df['fyear'] == year_start]
rows_start = rows_start.reset_index(drop=True)

rows_start = rows_start[rows_start['datadate'] <= int(f'{year_start if only_tickers_with_fiscal_year_ending_at <= 12 else year_start+1}{only_tickers_with_fiscal_year_ending_at}31')]

tickers_start = rows_start['tic']
tickers_start = set(tickers_start)
print(f'{len(tickers_start)} tickers in {year_start}')

df_columns_start = most_frequent_columns(rows_start)
path = f'./columns_{year_start}.csv'
df_columns_start.to_csv(path, index=False)
df_columns_start

3863 tickers in 2016


,Column,Count,Description,FileTag
0,gvkey,4506.0,Global Company Key - Key Selection,Multiple Files
1,datadate,4506.0,Data Date,Multiple Files
2,fyear,4506.0,Data Year - Fiscal,DES_ANN
3,indfmt,4506.0,Not found,Not found
4,consol,4506.0,Level of Consolidation,CCM_FILEDATE
...,...,...,...,...
976,xoptd,0.0,Implied Option EPS Diluted,IS_ANN_INDL
977,xopteps,0.0,Implied Option EPS Basic,IS_ANN_INDL
978,add3,0.0,Address Line 3,COMPANY
979,add4,0.0,Address Line 4,COMPANY


In [18]:
min_tickers_to_select_row = (4500/4740) * len(rows_start['tic'])
print(min_tickers_to_select_row)
# select columns with count > min_count
def select_columns(df, min_count, file_tag):
    vals = df[df_columns_start['FileTag'] == file_tag]
    vals = vals[vals['Count'] > min_count]['Column']
    return vals.values.tolist()

vals_bal_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'BAL_ANN_INDL')
vals_is_ann_indl = select_columns(df_columns_start, min_tickers_to_select_row, 'IS_ANN_INDL')

print(f'{len(vals_bal_ann_indl)} columns in BAL_ANN_INDL')
print(f'{len(vals_is_ann_indl)} columns in IS_ANN_INDL')

# select all rows in rows_start with columns of vals_bal_ann_indl and vals_is_ann_indl different than NaN
rows_start_bal_and_is = rows_start[['tic', 'fyear'] + vals_bal_ann_indl + vals_is_ann_indl + ['mkvalt', 'prcc_f']]
rows_start_bal_and_is = rows_start_bal_and_is.dropna()
rows_start_bal_and_is = rows_start_bal_and_is.reset_index(drop=True)
# print(len(rows_start_bal_and_is))
print(f'{len(rows_start_bal_and_is)} rows with all columns in {year_start}')

# create csv file
path = f'./rows_{year_start}_bal_ann_indl.csv'
rows_start_bal_and_is.to_csv(path, index=False)

tickers = rows_start_bal_and_is['tic']

data = rows_start_bal_and_is
data

4277.848101265822
21 columns in BAL_ANN_INDL
9 columns in IS_ANN_INDL
3661 rows with all columns in 2016


,tic,fyear,ao,at,icapt,lse,pstkr,tstkp,ceq,dltt,...,do,dvp,ib,revt,xido,dvc,dvt,ibmii,mkvalt,prcc_f
0,AAL,2016.0,3527.000,51274.000,26274.000,51274.000,0.0,0.0,3785.000,22489.000,...,0.0,0.0,2676.000,40180.000,0.0,224.000,224.000,2676.000,23685.5569,46.69
1,CECO,2016.0,5.463,498.634,312.449,498.634,0.0,0.0,190.082,122.367,...,0.0,0.0,-38.218,417.011,0.0,8.995,8.995,-38.254,476.5599,13.95
2,PRG,2016.0,33.659,2615.736,1832.912,2615.736,0.0,0.0,1481.598,351.314,...,0.0,0.0,139.283,3207.716,0.0,7.420,7.420,139.283,2285.6535,31.99
3,ABT,2016.0,5016.000,52666.000,41398.000,52666.000,0.0,0.0,20538.000,20681.000,...,337.0,0.0,1063.000,20853.000,337.0,1547.000,1547.000,1083.000,56572.8983,38.41
4,ACU,2016.0,0.694,92.066,79.056,92.066,0.0,0.0,46.120,32.936,...,0.0,0.0,5.851,124.574,0.0,1.330,1.330,5.851,85.0203,25.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3656,NORD,2016.0,109.600,2368.100,1413.400,2368.100,0.0,0.0,286.500,1121.500,...,0.0,0.0,47.100,856.000,0.0,0.000,0.000,49.200,2205.3040,21.18
3657,ESI,2016.0,104.500,10054.100,8012.700,10054.100,0.0,0.0,2736.100,5122.900,...,0.0,0.0,-73.700,3585.900,0.0,0.000,0.000,-76.700,2788.2080,9.81
3658,NETI,2016.0,6.357,1547.157,1522.607,1547.157,0.0,0.0,956.615,565.992,...,0.0,0.0,-124.835,78.402,0.0,0.000,0.000,-124.835,380.2600,5.05
3659,NAO.3,2016.0,0.000,374.854,370.389,374.854,0.0,0.0,234.196,136.193,...,0.0,0.0,-32.151,17.697,0.0,5.997,5.997,-32.151,56.8893,2.75


#### Regressor

In [31]:
#mlp regressor 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.kernel_ridge import KernelRidge



import numpy as np

# split in x and y
x = data.drop(columns=['tic', 'fyear', 'mkvalt', 'prcc_f'])
y = data['mkvalt']

# normalize x and y
scaler = StandardScaler()
x = scaler.fit_transform(x)
# y = scaler.fit_transform(y.values.reshape(-1, 1))   

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# # Linear Regression
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(x_train, y_train)
# score = regressor.score(x_test, y_test)
# print(f'Linear Regression Score: {score}')

# # Gradient Boosting Regressor
# from sklearn.ensemble import GradientBoostingRegressor
# regressor = GradientBoostingRegressor(
#     n_estimators=50,
#     learning_rate=0.1,
#     max_depth=3,
#     loss='ls'
# )
# regressor.fit(x_train, y_train)
# score = regressor.score(x_test, y_test)
# print(f'Gradient Boosting Regressor Score: {score}')

# # Random Forest Regressor
# from sklearn.ensemble import RandomForestRegressor
# regressor = RandomForestRegressor(
#     n_estimators=100,
#     max_depth=10,
#     random_state=42
# )
# regressor.fit(x_train, y_train)
# score = regressor.score(x_test, y_test)
# print(f'Random Forest Regressor Score: {score}')

# # KernelRidge Regressor
# regressor = KernelRidge(
#     alpha=1.0,
#     kernel='linear',
#     gamma=None,
#     degree=3,
#     coef0=1,
#     kernel_params=None
# )
# regressor.fit(x_train, y_train)
# score = regressor.score(x_test, y_test)
# print(f'KernelRidge Regressor Score: {score}')

# MLP Regressor
mlp_regressor = MLPRegressor(
    learning_rate='adaptive',
    max_iter=1000,
    hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100),
    activation='relu',
    solver='adam',
)

mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)
print(f'MLP Regressor Test Score: {score}')

# mlp_regressor.fit(x, y)
# score = mlp_regressor.score(x, y)
# print(f'MLP Regressor Training Score: {score}')


MLP Regressor Test Score: 0.8287266742451336


#### Simulation

In [32]:


print(f'Analysing {len(data)} tickers')
mapT = {}

market_caps_2021 = market_caps_for_year(year_end)
market_caps_2020 = market_caps_for_year(year_start)

# loop through data
for i in range(len(data)):
    row = data.iloc[i]
    ticker = row['tic']
    x_i = x[i]
    # print(x_i)
    y_i = row['mkvalt']
    # mlp_regressor.predict([x_i])
    p = mlp_regressor.predict([x_i])
    diff = p[0]/y_i - 1
    # print(f'{ticker}: {p} / ${y_i} ({diff})\n')
    if diff > 0:
        mapT[ticker] = diff

# sort mapT by biggest diff
mapT = {k: v for k, v in sorted(mapT.items(), key=lambda item: item[1], reverse=True)}

#get top 30
top30 = list(mapT.keys())[:30]

count = 0
sumAll = 0
sumAllYf = 0
sumAllYfJune = 0
sumAllYfMPlus3 = 0
noyf = []

for tic in top30:
    vals_2021 = market_caps_2021[market_caps_2021['tic'] == tic]['price'].values
    # we also exclude some tickers here that are presenting a huge change in market cap - they don't have historical prices for 2020 for some reason, doesn't seem like a good idea to use them
    if len(vals_2021) == 0:
        continue
    m2021 = vals_2021[0] 
    m2020 = market_caps_2020[market_caps_2020['tic'] == tic]['price'].values[0] 
    change = m2021/m2020-1
    exchg = df[df['tic'] == tic]['exchg'].values[0]
    print(f'{tic}: {mapT[tic]} => {change} ({exchg})')
    yf2021 =  val_for_tic_at_year(tic, year_end)
    yf2020 = val_for_tic_at_year(tic, year_start)
    yf2021June = val_for_tic_at_year(tic, year_end, month=18)
    yf2020June = val_for_tic_at_year(tic, year_start, month=18)
    yf2021MPlus3 = val_for_tic_at_year(tic, year_end, month=only_tickers_with_fiscal_year_ending_at+3)
    yf2020MPlus3 = val_for_tic_at_year(tic, year_start, month=only_tickers_with_fiscal_year_ending_at+3)
    if not (yf2021 == None or yf2020 == None):
        changeYf = yf2021/yf2020-1
        sumAllYf += changeYf
        print(f'Yahoo Finance: {changeYf}')
    if not (yf2021June == None or yf2020June == None):
        sumAllYfJune += yf2021June/yf2020June-1
    if not (yf2021MPlus3 == None or yf2020MPlus3 == None):
        sumAllYfMPlus3 += yf2021MPlus3/yf2020MPlus3-1
        print(f'Yahoo Finance M+3: {yf2021MPlus3/yf2020MPlus3-1}')
    else:
        noyf.append(tic)

    sumAll += change
    count += 1
    print()

change = (sumAll*100)/count
changeYf = (sumAllYf*100)/(count - len(noyf))
changeYfJune = (sumAllYfJune)*100/(count - len(noyf))

print(f'Average change: {change:.2f}%')
print(f'Average change yf: {changeYf:.2f}%')
print(f'Average change yf (Investing only in june after all the fillings have been made public): {changeYfJune:.2f}%')
print(f'Average change yf M+3: {(sumAllYfMPlus3*100)/(count - len(noyf)):.2f}%')
print(f'No yf: {noyf}')

Analysing 3661 tickers


<ipython-input-14-c6952f2020db>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']
<ipython-input-14-c6952f2020db>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfyear['price'] = dfyear['prcc_f'] / dfyear['ajex']


VRME: 99.51779830113817 => 1.4545454545454546 (14.0)
Yahoo Finance: 2.5159361923488914
Yahoo Finance M+3: 1.324999968210856

PINC: 80.13248558279837 => 0.10091743119266039 (14.0)
Yahoo Finance: 0.1009174810706428
Yahoo Finance M+3: -0.03481797038714607

WAVD: 67.71295805670495 => 2.5 (14.0)
Yahoo Finance: 1.9999998935631358
Yahoo Finance M+3: 1.2499999068677439

EP: 66.18966857939243 => 0.0012698412698413097 (12.0)
Yahoo Finance: 0.02564098155475092
Yahoo Finance M+3: -0.06666669978035689

AIKI: 58.28424780371755 => 0.3461538461538458 (14.0)
- AIKI: No data found, symbol may be delisted
- AIKI: No data found, symbol may be delisted
- AIKI: No data found, symbol may be delisted
- AIKI: No data found, symbol may be delisted
- AIKI: No data found, symbol may be delisted
- AIKI: No data found, symbol may be delisted

RMBL: 57.75924601227551 => 20.204081632653065 (14.0)
Yahoo Finance M+3: 0.1428571428571428

ASTI: 53.51402318009507 => -0.6896551724137931 (14.0)

CMLS: 52.38049937379858 => -

<ipython-input-32-7266574b268f>:41: RuntimeWarning: invalid value encountered in double_scalars
  change = m2021/m2020-1



GOVX: 25.836757362567155 => nan (14.0)


...


VRS: 24.16131900409196 => 1.4746478873239437 (11.0)
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted
- VRS: No data found, symbol may be delisted

NSPR: 23.859157376627202 => nan (14.0)


<ipython-input-32-7266574b268f>:41: RuntimeWarning: invalid value encountered in double_scalars
  change = m2021/m2020-1


Yahoo Finance: -0.9449152542372882
Yahoo Finance M+3: 0.18811881188118806

DCTH: 22.94330441962288 => nan (14.0)


<ipython-input-32-7266574b268f>:41: RuntimeWarning: invalid value encountered in double_scalars
  change = m2021/m2020-1



IHRT: 22.191956066835193 => -0.5495495495495496 (14.0)

POCI: 22.137576940103088 => 0.13207547169811296 (14.0)
Yahoo Finance: 0.13207541794288713
Yahoo Finance M+3: 0.3749999254941969

SCWO: 21.336221129416426 => -0.07692307692307698 (14.0)
Yahoo Finance: 0.0
Yahoo Finance M+3: 0.0

SMTI: 21.15600952807835 => 0.5831134564643798 (14.0)
Yahoo Finance: 0.5
Yahoo Finance M+3: -0.33333333333333337

GBR: 21.061454396287505 => -0.27833333333333343 (12.0)
Yahoo Finance: -0.2238806218004019
Yahoo Finance M+3: -0.24137928435786404

OPHC: 20.919370385642907 => 0.24338624338624348 (14.0)
Yahoo Finance: 0.3153638987233873
Yahoo Finance M+3: 0.3933332761128743

TGC: 20.531080768904676 => 0.14285714285714302 (12.0)
- TGC: No data found, symbol may be delisted
- TGC: No data found, symbol may be delisted
- TGC: No data found, symbol may be delisted
- TGC: No data found, symbol may be delisted
- TGC: No data found, symbol may be delisted
- TGC: No data found, symbol may be delisted

NUZE: 19.886213786

In [33]:
columns = data.columns.values

for c in columns:
    print(get_description(c))

Historical Ticker/Trading Symbol
Data Year - Fiscal
Assets - Other
Assets - Total
Invested Capital - Total
Liabilities & Stockholders Equity - Total
Preferred/Preference Stock - Redeemable
Treasury Stock - Preferrred
Common/Ordinary Equity - Total
Long-Term Debt - Total
Preferred/Preference Stock - Nonredeemable
Treasury Stock - Common
Debt in Current Liabilities - Total
Goodwill
Intangible Assets - Total
Cash
Short-Term Investments - Total
Employees
Other Intangibles
Common/Ordinary Stock (Capital)
Capital Surplus/Share Premium Reserve
Long-Term Debt Due in One Year
Deferred Taxes (Balance Sheet)
Com Stk Equiv - Dollar Savings
Discontinued Operations
Dividends - Preferred/Preference
Income Bef Extraord Items
Revenue - Total
Extraord Items & Discontinued Operations
Dividends Common/Ordinary
Dividends - Total
Income Bef Extraord Items & Noncontrolling Interests
Market Value - Total
Not found
